In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import wandb

api = wandb.Api()


# set theme
sns.set_theme(
    context="paper",
    style="ticks",
    font_scale=0.8,
    rc={
        "figure.figsize": (3.5, 2.0),
        "figure.dpi": 300,
        "savefig.dpi": 300,
        "text.usetex": True,
        "lines.linewidth": 0.7,
        "axes.linewidth": 0.7,
        "axes.grid": True,
        "grid.linestyle": "--",
        "grid.linewidth": 0.5,
        "pdf.fonttype": 42,
    },
)

fig_path = Path("../figures")

# Comparison
Compare the performance in terms of coverage and number of collisions.

In [ ]:
models = {
    "c": "LSAP",
    "1zgs74or": "GNN (IL)",
    "d0": "0-Hop",
    "capt": "CAPT",
    "lo49pixb": "GNN (RL)",
    "d1": "1-Hop",
}
df = pd.concat(
    [
        pd.read_parquet(f"../data/test_results/{key}/{key}.parquet").assign(
            policy=policy, time=lambda x: x["step"] * 0.1
        )
        for key, policy in models.items()
    ]
)
# df["collisions"] /= 200
# df["near_collisions"] *= 200

In [ ]:
df["discount"] = 0.99 ** df["step"]
df_comparison = (
    (
        df.groupby(["policy", "trial"])
        .apply(
            lambda df: np.average(df["coverage"], weights=df["discount"]),
            include_groups=False,
        )
        .rename("discounted_coverage")  # type: ignore
        .to_frame()
    )
    .join(df.groupby(["policy", "trial"])[["collisions", "near_collisions"]].sum())
    .groupby("policy")
    .mean()
    .style.format("{:.2f}")
)
print(df_comparison.to_latex())
df_comparison

In [ ]:
# Create the line plot
plt.figure()
g = sns.lineplot(
    data=df,
    x="time",
    y="coverage",
    hue="policy",
    errorbar="sd",
    palette="deep",
    linewidth=0.8,
)
sns.move_legend(g, "lower right", ncol=2, title="Policy")
plt.ylim(0.1, 1.05)
plt.xlabel("Time")
plt.ylabel("Success Rate")
plt.savefig(fig_path / "coverage_comparison.pdf", bbox_inches="tight")
plt.show()

# Scalability Experiments
Vary the number of agents and their density. Report the discounted coverage. 

In [ ]:
df = pd.read_parquet("../data/scalability.parquet").sort_values(
    ["n_agents", "area"], ascending=False
)
df["density"] = (df["n_agents"] / df["area"]).round(1).apply(lambda x: f"{x:.1f}")
df["time"] = df["step"] * 0.1

## Only vary the number of agents, keep density at 1.0

In [ ]:
# Number of Agents at Density = 1.0 agents per meter squared
plt.figure()
g = sns.lineplot(
    data=(
        df.assign(n_agents=df["n_agents"].apply(lambda x: f"{x:d}")).query(
            "density=='1.0'"
        )
    ),
    x="time",
    y="coverage",
    hue="n_agents",
    palette="rocket_r",
    linewidth=0.8,
    errorbar=None,
)
sns.move_legend(g, "lower right", ncol=1, title="\# of Agents")
plt.ylim(0.50, 1.0)
plt.xlabel("Time")
plt.ylabel("Success Rate")
# plt.legend(loc="upper center", bbox_to_anchor=(0.5, -0.15), ncol=3, frameon=False)
plt.savefig(fig_path / "generalize_scale.pdf", bbox_inches="tight")
plt.show()

In [ ]:
collisions = (
    df.query("density=='1.0'")
    .groupby(["n_agents", "trial"])["collisions"]
    .sum()
    .groupby("n_agents")
    .mean()
)
# normalize to number of agents
collisions / collisions.index * 100

## Vary the density only, n_agents = 100

In [ ]:
# Number of Agents at Density = 1.0 agents per meter squared
plt.figure()
g = sns.lineplot(
    data=df.query("n_agents==100").sort_values("area", ascending=True),
    x="time",
    y="coverage",
    hue="density",
    style="density",
    palette="mako",
    linewidth=0.8,
    errorbar="sd",
)
sns.move_legend(g, "lower right", ncol=1, title="Agent Density")
plt.ylim(0.5, 1.0)
plt.xlabel("Time")
plt.ylabel("Success Rate")
# plt.legend(loc="upper center", bbox_to_anchor=(0.5, -0.15), ncol=3, frameon=False)
plt.savefig(fig_path / "generalize_density.pdf", bbox_inches="tight")
plt.show()

In [ ]:
df.query("n_agents==100").groupby(["density", "trial"])["collisions"].sum().groupby(
    "density"
).mean()

## Now vary both number of agents and density, report coverage

In [ ]:
gamma = 0.99
df["discount"] = gamma ** df["step"]

data: pd.DataFrame = (
    df.groupby(["n_agents", "density", "trial"])
    .apply(
        lambda df: np.average(df["coverage"], weights=df["discount"]),
        include_groups=False,
    )
    .rename("discounted_coverage")  # type: ignore
    .to_frame()
    .sort_values("density", ascending=False)
)


# Number of Agents at Density = 1.0 agents per meter squared
plt.figure()
g = sns.lineplot(
    data=data,
    x="n_agents",
    y="discounted_coverage",
    hue="density",
    style="density",
    markers=".",
    errorbar="sd",
    palette="mako",
)
sns.move_legend(g, "lower right", title="Agent Density")
# plt.ylim(0.80, 0.95)
plt.xlabel("\# of Agents")
plt.ylabel("Average Discounted Success Rate")
plt.savefig(fig_path / "generalize_scale_density.pdf", bbox_inches="tight")
plt.show()

In [ ]:
data: pd.DataFrame = (
    df.groupby(["n_agents", "density", "trial"])["collisions"]
    .sum()
    .groupby(["n_agents", "density"])
    .mean()
    .reset_index()
    .pivot(index="density", columns="n_agents", values="collisions")
)
print(data.style.format("{:.2f}").to_latex())

In [ ]:
data = (
    df.groupby(["n_agents", "density", "trial"])["collisions"]
    .sum()
    .groupby(["n_agents", "density"])
    .mean()
    .reset_index()
)
data["collisions"] *= 100 / data["n_agents"]
print(data.pivot(index="density", columns="n_agents").to_latex(float_format="%0.2f"))

In [ ]:
for id, name in zip(
    ["wh6kuxhs", "9pmses8z"], ["imitation_learning", "reinforcement_learning"]
):

    run = api.run(f"test-team-12/motion-planning/{id}")
    columns = ["val/coverage", "val/reward", "val/n_collisions", "epoch"]
    df_run = run.history(keys=columns)[columns]
    # apply a rolling window
    df_run = df_run.set_index("epoch").rolling(9).mean().reset_index(drop=False)

    palette = sns.color_palette("deep")

    plt.figure()
    sns.lineplot(
        data=df_run.melt(
            id_vars="epoch", value_vars=["val/coverage", "val/n_collisions"]
        ),
        x="epoch",
        y="value",
        hue="variable",
        errorbar=None,
    )
    plt.legend(["Success Rate", "Collisions"], title=None, loc="center right")
    plt.ylabel("Success Rate / Collisions")
    plt.xlabel("Epoch")
    plt.ylim(0, 1.0)

    plt.savefig(fig_path / f"{name}.pdf", bbox_inches="tight")
    plt.show()